In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from pso import ParticleSwarmOptimizedClustering
from particle import quantization_error, calc_sse
from utils import normalize
import utils
from kmeans import KMeans
from sklearn.metrics import silhouette_score

In [3]:
data = pd.read_csv('./data/iris.data', sep=',', header=None)
data.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
data_classes = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
y = np.array(data[4].apply(data_classes.index))
x = data.drop([4], axis=1)
x = x.values
x = normalize(x)

# K-Means

In [5]:
kmeans = KMeans(n_cluster=3, init_pp=False, seed=2018)
kmeans.fit(x)

In [6]:
predicted_kmeans = kmeans.predict(x)
print('Silhouette:', silhouette_score(x, predicted_kmeans))
print('SSE:', kmeans.SSE)
print('Quantization:', quantization_error(centroids=kmeans.centroid, data=x, labels=predicted_kmeans))
print('Err_ratio:', utils.stupid_precision_ratio(predicted_kmeans, y, 3))

Silhouette: 0.5043188549150883
SSE: 6.998114004826762
Quantization: 0.19524413664147766
Err_ratio: (0.8866666666666667, (2, 0, 1))


In [7]:
kmeans2 = KMeans(n_cluster=3, init_pp=True, seed=2018)
kmeans2.fit(x)
predicted_kmeans2 = kmeans2.predict(x)
print('Silhouette:', silhouette_score(x, predicted_kmeans))
print('SSE:', kmeans2.SSE)
print('Quantization:', quantization_error(centroids=kmeans2.centroid, data=x, labels=predicted_kmeans2))
print('Err_ratio:', utils.stupid_precision_ratio(predicted_kmeans2, y, 3))

Silhouette: 0.5043188549150883
SSE: 10.90827498962253
Quantization: 0.2049806792871817
Err_ratio: (0.5733333333333334, (2, 0, 1))


# PSO

In [8]:
pso = ParticleSwarmOptimizedClustering(
        n_cluster=3, n_particles=20,dump_w=0.99,w=1,c2=2,c1=2, data=x, hybrid=True, max_iter=2000, print_debug=50)

C:\Users\moham\Desktop\2CS\Optim\Projet\Partitioning_Clustering_using_PSO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


In [9]:
hist = pso.run()

Initial global best score 0.1873742297575847
Iteration 0001/2000 current gbest score 0.187374229757584693
Iteration 0051/2000 current gbest score 0.184819652656180611
Iteration 0101/2000 current gbest score 0.176104560521358561
Iteration 0151/2000 current gbest score 0.174530109491833202
Iteration 0201/2000 current gbest score 0.174527694581757431
Iteration 0251/2000 current gbest score 0.174527674176354064
Iteration 0301/2000 current gbest score 0.174527673192589189
Iteration 0351/2000 current gbest score 0.174527673191308103
Iteration 0401/2000 current gbest score 0.174527673191305771
Iteration 0451/2000 current gbest score 0.174527673191305688
Iteration 0501/2000 current gbest score 0.174527673191305688
Iteration 0551/2000 current gbest score 0.174527673191305688
Iteration 0601/2000 current gbest score 0.174527673191305688
Iteration 0651/2000 current gbest score 0.174527673191305688
Iteration 0701/2000 current gbest score 0.174527673191305688
Iteration 0751/2000 current gbest score 

In [11]:
pso_kmeans = KMeans(n_cluster=3, init_pp=False, seed=2018)

In [12]:
pso_kmeans.centroid = pso.gbest_centroids.copy()
pso_kmeans.centroid

array([[0.53204151, 0.35645623, 0.65311189, 0.6460833 ],
       [0.23341459, 0.6648297 , 0.09420586, 0.0731431 ],
       [0.0820042 , 0.4686312 , 0.05418396, 0.05670269]])

In [13]:
predicted_pso = pso_kmeans.predict(x)
print('Silhouette:', silhouette_score(x, predicted_pso))
print('SSE:', calc_sse(centroids=pso.gbest_centroids, data=x, labels=predicted_pso))
print('Quantization:', pso.gbest_score)
print('Err_ratio:', utils.stupid_precision_ratio(predicted_pso, y, 3))

Silhouette: 0.5052462651098638
SSE: 11.307346265753136
Quantization: 0.17452767318949433
Err_ratio: (0.54, (1, 0, 2))


In [14]:
predicted_pso

array([1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1,
       1, 2, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [15]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

# Repeated Test

### K-Means++

In [13]:
kmeanspp = {
    'silhouette': [],
    'sse' : [],
    'quantization' : [],
}
for _ in range(20):
    kmean_rep = KMeans(n_cluster=3, init_pp=True)
    kmean_rep.fit(x)
    predicted_kmean_rep = kmean_rep.predict(x)
    silhouette = silhouette_score(x, predicted_kmean_rep)
    sse = kmean_rep.SSE
    quantization = quantization_error(centroids=kmean_rep.centroid, data=x, labels=predicted_kmean_rep)
    kmeanspp['silhouette'].append(silhouette)
    kmeanspp['sse'].append(sse)
    kmeanspp['quantization'].append(quantization)

### PSO 

In [19]:
%%time
pso_plain = {
    'silhouette': [],
    'sse' : [],
    'quantization' : [],
}
for _ in range(20):
    pso_rep = ParticleSwarmOptimizedClustering(
        n_cluster=3, n_particles=10, data=x,dump_w=0.99,c1=2,c2=2,w=1, hybrid=False, max_iter=2000, print_debug=2000)
    pso_rep.run()
    pso_kmeans = KMeans(n_cluster=3, init_pp=False, seed=2018)
    pso_kmeans.centroid = pso_rep.gbest_centroids.copy()
    predicted_pso_rep = pso_kmeans.predict(x)
    
    silhouette = silhouette_score(x, predicted_pso_rep)
    sse = calc_sse(centroids=pso_rep.gbest_centroids, data=x, labels=predicted_pso_rep)
    quantization = pso_rep.gbest_score
    pso_plain['silhouette'].append(silhouette)
    pso_plain['sse'].append(sse)
    pso_plain['quantization'].append(quantization)

Initial global best score 0.2431153843813657
Iteration 0001/2000 current gbest score 0.243115384381365707


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 0.213098205576494171
Initial global best score 0.19467141182285472
Iteration 0001/2000 current gbest score 0.194671411822854717


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 0.179239468351963444
Initial global best score 0.18507380261010256
Iteration 0001/2000 current gbest score 0.185073802610102561


...

Finish with gbest score 0.189492063922788473
Initial global best score 0.2269481653403105
Iteration 0001/2000 current gbest score 0.226948165340310509


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 0.201712695984712531
Initial global best score 0.23984638791186433
Iteration 0001/2000 current gbest score 0.237337059875291440


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 0.198277540932677659
Wall time: 2min 11s


### PSO Hybrid

In [21]:
%%time
pso_hybrid = {
    'silhouette': [],
    'sse' : [],
    'quantization' : [],
}
for _ in range(20):
    pso_rep = ParticleSwarmOptimizedClustering(
        n_cluster=3, n_particles=10, data=x,dump_w=0.99,c1=2,c2=2,w=1, hybrid=True, max_iter=2000, print_debug=2000)
    pso_rep.run()
    pso_kmeans = KMeans(n_cluster=3, init_pp=False, seed=2018)
    pso_kmeans.centroid = pso_rep.gbest_centroids.copy()
    predicted_pso_rep = pso_kmeans.predict(x)
    
    silhouette = silhouette_score(x, predicted_pso_rep)
    sse = calc_sse(centroids=pso_rep.gbest_centroids, data=x, labels=predicted_pso_rep)
    quantization = pso_rep.gbest_score
    pso_hybrid['silhouette'].append(silhouette)
    pso_hybrid['sse'].append(sse)
    pso_hybrid['quantization'].append(quantization)

Initial global best score 0.19524413664147766
Iteration 0001/2000 current gbest score 0.195244136641477661


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 0.194860482920887007
Initial global best score 0.19618667657047784
Iteration 0001/2000 current gbest score 0.196186676570477836


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 0.195831556995042927
Initial global best score 0.1962931514745171
Iteration 0001/2000 current gbest score 0.196293151474517091


...

Finish with gbest score 0.193876885159073725
Initial global best score 0.19524413664147766
Iteration 0001/2000 current gbest score 0.195244136641477661


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 0.194122351984317515
Initial global best score 0.1962931514745171
Iteration 0001/2000 current gbest score 0.196293151474517091


d:\study\2SIQ3\OPTIM\Partitioning_Clustering_using_ACO_metaheuristic\particle.py:14: RuntimeWarning: invalid value encountered in double_scalars
  dist /= len(idx)


Finish with gbest score 0.193231098421109637
Wall time: 1min 50s


# Comparison

In [22]:
benchmark = {
    'method' : ['K-Means++', 'PSO', 'PSO Hybrid'],
    'sse_mean' : [
        np.around(np.mean(kmeanspp['sse']), decimals=10),
        np.around(np.mean(pso_plain['sse']), decimals=10),
        np.around(np.mean(pso_hybrid['sse']), decimals=10),
    ],
    'sse_stdev' : [
        np.around(np.std(kmeanspp['sse']), decimals=10),
        np.around(np.std(pso_plain['sse']), decimals=10),
        np.around(np.std(pso_hybrid['sse']), decimals=10),
    ],
    'silhouette_mean' : [
        np.around(np.mean(kmeanspp['silhouette']), decimals=10),
        np.around(np.mean(pso_plain['silhouette']), decimals=10),
        np.around(np.mean(pso_hybrid['silhouette']), decimals=10),
    ],
    'silhouette_stdev' : [
        np.around(np.std(kmeanspp['silhouette']), decimals=10),
        np.around(np.std(pso_plain['silhouette']), decimals=10),
        np.around(np.std(pso_hybrid['silhouette']), decimals=10),
    ],
    'quantization_mean' : [
        np.around(np.mean(kmeanspp['quantization']), decimals=10),
        np.around(np.mean(pso_plain['quantization']), decimals=10),
        np.around(np.mean(pso_hybrid['quantization']), decimals=10),
    ],
    'quantization_stdev' : [
        np.around(np.std(kmeanspp['quantization']), decimals=10),
        np.around(np.std(pso_plain['quantization']), decimals=10),
        np.around(np.std(pso_hybrid['quantization']), decimals=10),
    ],
}

In [23]:
benchmark

{'method': ['K-Means++', 'PSO', 'PSO Hybrid'],
 'sse_mean': [8.0459211006, 10.3455890994, 7.9015546244],
 'sse_stdev': [1.6535763914, 2.3428745331, 1.596426723],
 'silhouette_mean': [0.4917327084, 0.4871404119, 0.4940488978],
 'silhouette_stdev': [0.0095544033, 0.0235880626, 0.0115855836],
 'quantization_mean': [0.1981495423, 0.1993579093, 0.1938878187],
 'quantization_stdev': [0.0039626853, 0.0131629743, 0.0069952424]}

In [31]:
benchmark_df = pd.DataFrame.from_dict(benchmark)
benchmark_df

,method,sse_mean,sse_stdev,silhouette_mean,silhouette_stdev,quantization_mean,quantization_stdev
0,K-Means++,8.045921,1.653576,0.491733,0.009554,0.198150,0.003963
1,PSO,10.345589,2.342875,0.487140,0.023588,0.199358,0.013163
2,PSO Hybrid,7.901555,1.596427,0.494049,0.011586,0.193888,0.006995


In [25]:
benchmark_df.to_excel('benchmark_iris_res.xlsx', index=False)

In [26]:
benchmark_df.to_csv('benchmark_iris_res.csv', index=False)